## Question 3: Image completion
We use patch match method for completing the given two images.
Functions are implemented in `q3_funcs`

## How the algorithm works?
Algorithm is rather similar to the original patch match method. Beginning with an
initial image, a reference image and a given patch size, in each iteration, we perform
propagation and random search on the image and update main image at the end of the iteration.
In the main image, we left a margin of patch size on the right and bottom unchanged,
and for updating the value of a pixel, we just take the corresponding values on the reference
which are on the square with side length $\frac{patch size}{2}$ on the lefth-top of the pixel
and calculate their average. Functions used in each step are described below :

## q3_funcs
First functions generates a random offset matrix from given
image and reference and a patch size :

In [ ]:
import random

import numpy as np
import cv2
import q2_funcs as q2

''' patch match method for image completion '''

def initialize_random(A, B, patch_size):
    A_dims, B_dims = A.shape, B.shape
    offsets_x = np.zeros((A_dims[0] - patch_size[0], A_dims[1] - patch_size[1]), dtype='int16')
    offsets_y = np.zeros((A_dims[0] - patch_size[0], A_dims[1] - patch_size[1]), dtype='int16')
    for i in range(A_dims[0] - patch_size[0]):
        for j in range(A_dims[1] - patch_size[1]):
            offsets_x[i, j] = np.random.randint(-i, B_dims[0] - i - patch_size[0] - 1)
            offsets_y[i, j] = np.random.randint(-j, B_dims[1] - j - patch_size[1] - 1)
    offsets = np.zeros((A_dims[0] - patch_size[0], A_dims[1] - patch_size[1], 2), dtype='int16')
    offsets[:, :, 0] = offsets_x.copy()
    offsets[:, :, 1] = offsets_y.copy()
    return offsets

Next function is used to find the difference of the patch of a pixel
in main image with its patch in reference image. Method used for
getting difference is SSD :


In [ ]:
def find_difference(A, B, offsets, patch_size, x, y):
    patch_A = A[x:x + patch_size[0], y:y + patch_size[1], :].copy()
    patch_B = B[x + int(offsets[x, y, 0]):x + int(offsets[x, y, 0]) + patch_size[0],
              y + int(offsets[x, y, 1]):y + int(offsets[x, y, 1]) + patch_size[1], :].copy()
    ssd = np.sum(np.square(patch_A - patch_B))
    return ssd

function `get_random_patch` generates the coordinates of a
random patch in reference image from a given boundary.
It is being used in `random_search` function :

In [ ]:
def get_random_patch(B, x_min, x_max, y_min, y_max, patch_size):
    high_x, high_y = x_max - patch_size[0], y_max - patch_size[1]
    x, y = np.random.randint(x_min, high_x - 1), np.random.randint(y_min, high_y - 1)
    return x, y

Next function performs the random search step of patch match,
with given x,y coordinates of pixel in main image, and difference
calculated of neighbor pixels patch :

In [ ]:
def random_search(A, B, offsets, patch_size, x, y, d, alpha):
    patch_A = A[x:x + patch_size[0], y:y + patch_size[1], :].copy()
    x_b, y_b = x + offsets[x, y, 0], y + offsets[x, y, 1]
    offset_final = offsets[x, y].copy()
    d_min = d
    # parameters indicating window :
    x_min, x_max = 0, B.shape[0] - 1
    y_min, y_max = 0, B.shape[1] - 1
    while (x_max - x_min) > 2 * patch_size[0] and (y_max - y_min) > 2 * patch_size[1]:
        x_r, y_r = get_random_patch(B, x_min, x_max, y_min, y_max, patch_size)
        random_patch = B[x_r:x_r + patch_size[0], y_r:y_r + patch_size[1], :].copy()
        ssd = np.sum(np.square(patch_A - random_patch))
        if ssd < d_min:
            d_min = ssd
            offset_final = (x_r - x, y_r - y)
        # updating window
        x_min, x_max = x_min + int((x_b - x_min) / alpha), x_max - int((x_max - x_min - patch_size[0]) / alpha)
        y_min, y_max = y_min + int((y_b - y_min) / alpha), y_max - int((y_max - y_min - patch_size[1]) / alpha)
    return offset_final

Window is initially the whole reference, and in each step, decreases
with rate alpha.

`update_value` function is used to calculate the value of a
pixel in main image from reference and offsets. Coordinates of
pixel is given in x,y:

In [ ]:
def update_value(A, B, offsets, patch_size, x, y):
    values_x = offsets[max(0, x - int(patch_size[0] / 2) + 1):x + 1, max(0, y - int(patch_size[1] / 2) + 1): y + 1,
               0] + x
    values_y = offsets[max(0, x - int(patch_size[0] / 2) + 1):x + 1, max(0, y - int(patch_size[1] / 2) + 1): y + 1,
               1] + y
    values_x = values_x.reshape(-1).astype('int16')
    values_y = values_y.reshape(-1).astype('int16')
    value = np.array(
        [np.sum(B[values_x, values_y, 0]), np.sum(B[values_x, values_y, 1]), np.sum(B[values_x, values_y, 2])]) / (
                values_x.shape[0])
    A[x, y] = value

The value is calculated by taking an average over the corresponding
values of the pixel in neighbor pixels patches in reference.

Two function `propagation_odd` and `propagation_even` perform
propagation step for odd and even iterations. implementation is
just in the way used in original patch match :

In [ ]:
def propagate_odd(A, B, offsets, patch_size, alpha=2):
    m, n = A.shape[0] - patch_size[0], A.shape[1] - patch_size[1]
    for i in range(m):
        for j in range(n):
            if i == 0:  # on the first row
                if j > 0:
                    d1, d2 = find_difference(A, B, offsets, patch_size, i, j), find_difference(A, B, offsets,
                                                                                               patch_size, i, j - 1)
                    d = min(d1, d2)
                    if d == d2:
                        if offsets[i, j - 1, 1] + j < B.shape[1] - patch_size[1]:
                            offsets[i, j] = (offsets[i, j - 1, 0], offsets[i, j - 1, 1])
                        else:
                            offsets[i, j] = (offsets[i, j - 1, 0], offsets[i, j - 1, 1] - 1)
                    offset_final = random_search(A, B, offsets, patch_size, i, j, d, alpha)
                    offsets[i, j] = offset_final
                    # update_value(A, B, offsets, patch_size, i, j)
            else:
                if j > 0:  # inside the matrix
                    d1 = find_difference(A, B, offsets, patch_size, i, j)
                    d2 = find_difference(A, B, offsets, patch_size, i - 1, j)
                    d3 = find_difference(A, B, offsets, patch_size, i, j - 1)
                    d = min(d1, d2, d3)
                    if d == d2:
                        if offsets[i - 1, j, 0] + i < B.shape[0] - patch_size[0]:
                            offsets[i, j] = (offsets[i - 1, j, 0], offsets[i - 1, j, 1])
                        else:
                            offsets[i, j] = (offsets[i - 1, j, 0] - 1, offsets[i - 1, j, 1])
                    elif d == d3:
                        if offsets[i, j - 1, 1] + j < B.shape[1] - patch_size[1]:
                            offsets[i, j] = (offsets[i, j - 1, 0], offsets[i, j - 1, 1])
                        else:
                            offsets[i, j] = (offsets[i, j - 1, 0], offsets[i, j - 1, 1] - 1)
                else:  # on the first column
                    d1, d2 = find_difference(A, B, offsets, patch_size, i, j), find_difference(A, B, offsets,
                                                                                               patch_size, i - 1, j)
                    d = min(d1, d2)
                    if d == d2:
                        if offsets[i - 1, j, 0] + i < B.shape[0] - patch_size[0]:
                            offsets[i, j] = (offsets[i - 1, j, 0], offsets[i - 1, j, 1])
                        else:
                            offsets[i, j] = (offsets[i - 1, j, 0] - 1, offsets[i - 1, j, 1])
                offset_final = random_search(A, B, offsets, patch_size, i, j, d, alpha)
                offsets[i, j] = offset_final
                # update_value(A, B, offsets, patch_size, i, j)


In [ ]:
def propagate_even(A, B, offsets, patch_size, alpha=2):
    m, n = A.shape[0] - patch_size[0], A.shape[1] - patch_size[1]
    for i in reversed(range(m)):
        for j in reversed(range(n)):
            if i == m - 1:  # on the last row
                if j < n - 1:
                    d1, d2 = find_difference(A, B, offsets, patch_size, i, j), find_difference(A, B, offsets,
                                                                                               patch_size, i, j + 1)
                    d = min(d1, d2)
                    if d == d2:
                        if offsets[i, j + 1, 1] + j >= 0:
                            offsets[i, j] = (offsets[i, j + 1, 0], offsets[i, j + 1, 1])
                        else:
                            offsets[i, j] = (offsets[i, j + 1, 0], offsets[i, j + 1, 1] + 1)
                    offset_final = random_search(A, B, offsets, patch_size, i, j, d, alpha)
                    offsets[i, j] = offset_final
                    # update_value(A, B, offsets, patch_size, i, j)
            else:
                if j < n - 1:  # inside the matrix
                    d1 = find_difference(A, B, offsets, patch_size, i, j)
                    d2 = find_difference(A, B, offsets, patch_size, i + 1, j)
                    d3 = find_difference(A, B, offsets, patch_size, i, j + 1)
                    d = min(d1, d2, d3)
                    if d == d2:
                        if offsets[i + 1, j, 0] + i >= 0:
                            offsets[i, j] = (offsets[i + 1, j, 0], offsets[i + 1, j, 1])
                        else:
                            offsets[i, j] = (offsets[i + 1, j, 0] + 1, offsets[i + 1, j, 1])
                    elif d == d3:
                        if offsets[i, j + 1, 1] + j >= 0:
                            offsets[i, j] = (offsets[i, j + 1, 0], offsets[i, j + 1, 1])
                        else:
                            offsets[i, j] = (offsets[i, j + 1, 0], offsets[i, j + 1, 1] + 1)
                else:  # on the last column
                    d1, d2 = find_difference(A, B, offsets, patch_size, i, j), find_difference(A, B, offsets,
                                                                                               patch_size, i + 1, j)
                    d = min(d1, d2)
                    if d == d2:
                        if offsets[i + 1, j, 0] + i >= 0:
                            offsets[i, j] = (offsets[i + 1, j, 0], offsets[i + 1, j, 1])
                        else:
                            offsets[i, j] = (offsets[i + 1, j, 0] + 1, offsets[i + 1, j, 1])
                offset_final = random_search(A, B, offsets, patch_size, i, j, d, alpha)
                offsets[i, j] = offset_final
                # update_value(A, B, offsets, patch_size, i, j)

function `generate_image` creates the whole image using reference and
offsets and `update_value` method:

In [ ]:
def generate_image(A, patch_size, offsets, B):
    m, n = offsets.shape[0], offsets.shape[1]
    for i in range(m):
        for j in range(n):
            update_value(A, B, offsets, patch_size, i, j)

Finally, `perform_patch` gets main image,reference and the number of iterations
and performs pathc match using the functions implemented before:

In [ ]:
def perform_patch_match(A,B,patch_size,iteration=4):
    offsets=initialize_random(A,B,patch_size)
    for i in range(iteration):
        propagate_odd(A, B, offsets, patch_size)
        generate_image(A, patch_size, offsets, B)

        propagate_even(A, B, offsets, patch_size)
        generate_image(A, patch_size, offsets, B)
    return A

The two final functions are for image completion using
texture synthesis, which is not used in this question ( and we won't
describe it )

In [ ]:
def synthesis_row_i(img, texture, patch_size, overlap, i):
    x = i * (patch_size[0] - overlap)
    mask = np.zeros((patch_size[0], patch_size[1], 3), dtype='uint8')
    mask[0:overlap, :, :] = 1
    mask[:, 0:overlap, :] = 1
    number_of_patches = int((img.shape[1] - patch_size[1]) / (patch_size[1] - overlap))
    for j in range(number_of_patches + 1):
        template_x = j * (patch_size[1] - overlap)
        template = img[x:x + patch_size[0], template_x:template_x + patch_size[1], :].copy()
        x1, y1 = q2.find_patch_from_L_template(texture[0:-patch_size[0], 0:-patch_size[1], :], template, mask)
        template2 = texture[x1:x1 + patch_size[1], y1:y1 + patch_size[0], :].copy()
        min_cut = q2.find_L_min_cut(template, template2, overlap)
        patch = (min_cut * template + (1 - min_cut) * template2).copy()
        img[x:x + patch_size[0], template_x:template_x + patch_size[1], :] = patch.copy()


def complete_image_with_texture(img, texture, patch_size, overlap):
    num_of_iterations = int(img.shape[0] / (patch_size[0] - overlap)) - 1
    for i in range(num_of_iterations):
        synthesis_row_i(img, texture, patch_size, overlap, i)

## Main Question code
In main file, we use previous functions to complete the image.
For first image (birds ) , some parts of the main image, with theier corresponding
reference parts are selected to be reconstructed with patch match.
In the second image ( the swimmer ) a rectangle containing the swimmer is selected ,
as well as the bottom part of the image ( as reference ) and patch match is applied.

In [ ]:
import numpy as np
import cv2
import q3_funcs

input_paths = ["inputs/im03-1.jpg", "inputs/im04.jpg"]

output_paths = ["outputs/res15.jpg", "outputs/res16.jpg"]

img_birds = cv2.imread(input_paths[0])
img_swimmer = cv2.imread(input_paths[1])

''' removing birds  '''

result_birds = img_birds.copy()
patch_size = (6, 6)
# two bottom birds
A1 = img_birds[700:950, 820:980, :].copy()
B1 = img_birds[720:950, 480:780, :].copy()
result1 = q3_funcs.perform_patch_match(A1, B1, patch_size, 4)
result_birds[700:950, 820:980, :] = result1

A2 = img_birds[610:740, 1130:1250, :].copy()
B2 = img_birds[550:650, 1110:1155, :].copy()
result2 = q3_funcs.perform_patch_match(A2, B2, patch_size, 4)
result_birds[610:740, 1130:1250, :] = result2

A3_1_2 = result_birds[120:175, 325:381, :].copy()
B3_1 = img_birds[140:240, 330:390, :].copy()
result3_1_2 = q3_funcs.perform_patch_match(A3_1_2, B3_1, patch_size, 4)
result_birds[120:175, 325:381, :] = result3_1_2

A3_1_1 = result_birds[65:175, 325:381, :].copy()
B3_1 = img_birds[140:240, 330:390, :].copy()
result3_1_1 = q3_funcs.perform_patch_match(A3_1_1, B3_1, patch_size, 4)
result_birds[65:175, 325:381, :] = result3_1_1


A3_2 = result_birds[65:175, 375:441, :].copy()
B3_2 = img_birds[80:165, 220:305, :].copy()
result3_2 = q3_funcs.perform_patch_match(A3_2, B3_2, patch_size, 4)
result_birds[65:175, 375:441, :] = result3_2

A3_3 = result_birds[65:175, 435:491, :].copy()
B3_3 = img_birds[120:240, 425:480, :].copy()
result3_3 = q3_funcs.perform_patch_match(A3_3, B3_3, patch_size, 4)
result_birds[65:175, 435:491, :] = result3_3

A3_4 = result_birds[65:175, 485:551, :].copy()
B3_4 = img_birds[140:270, 460:530, :].copy()
result3_4 = q3_funcs.perform_patch_match(A3_4, B3_4, patch_size, 4)
result_birds[65:175, 485:551, :] = result3_4

cv2.imwrite(output_paths[0], result_birds)

''' removing swimmer '''
patch_size = (6, 6)
A = img_swimmer[680:1200 + patch_size[0], 740:960 + patch_size[1], :].copy()
B = img_swimmer[1200:, :, :].copy()
offsets = q3_funcs.initialize_random(A, B, patch_size)
result = q3_funcs.perform_patch_match(A, B, patch_size, 4)
img_swimmer[680:1200 + patch_size[0], 740:960 + patch_size[1], :] = result
cv2.imwrite(output_paths[1], img_swimmer)

# result = q3_funcs.perform_patch_match(A, B, patch_size, 4)
# img_birds[50:170 + patch_size[0], 330:550 + patch_size[1], :] = result

A1 = A[:, 0:45, :].copy()
B1 = img_birds[140:190, 330:370, :].copy()

result = q3_funcs.perform_patch_match(A1, B1, patch_size, 4)
cv2.imwrite(output_paths[0], img_birds)

In [ ]:
patch_size = (6, 6)
A = img_swimmer[680:1200 + patch_size[0], 740:960 + patch_size[1], :].copy()
B = img_swimmer[1200:, :, :].copy()
offsets = q3_funcs.initialize_random(A, B, patch_size)
result=q3_funcs.perform_patch_match(A,B,patch_size,4)
img_swimmer[680:1200 + patch_size[0], 740:960 + patch_size[1], :] = result
cv2.imwrite(output_paths[1], img_swimmer)

Note : Although the main algorithm (patch match ) seems to work fine,
result of the first image is not very good; branches behind the birds
on the top left of the image are not reconstructed very well.
This might be happening because of selecting bad references or
not splitting this part properly.